In [1]:
# Imports
## Import data preparation libraries
import pandas as pd 
import numpy as np 
from datetime import datetime

##Import data enrichment library
import requests 

## Import database libraries 
from sqlalchemy import create_engine, Table, Column, Integer, String, Float, Boolean, Date, MetaData, ForeignKey
from sqlalchemy.orm import relationship, backref, sessionmaker
from sqlalchemy.ext.declarative import declarative_base

import locale
locale.setlocale(locale.LC_ALL, 'fr_FR.UTF-8')

'fr_FR.UTF-8'

In [ ]:
# choix à faire sur l'architecture base de données :
## soit transactions(avec base maisons et base appartements), soit adresses
iter_csv= pd.read_csv("../data/valeursfoncieres-2021.txt", sep='|',iterator=True, chunksize=100000, low_memory = False)
df_maison = pd.concat([chunk[chunk['Code type local'] == 1] for chunk in iter_csv]) 
df_maison.info()

In [ ]:
df_tot= pd.read_csv("../data/valeursfoncieres-2021.txt", sep='|', low_memory = False)

In [1]:
import pandas as pd


class get_data:
    """ Read data from csv and load it in a dataframe
    accepted arguments : path to file , separator, chunksize and filter
    option to load csv by filtering on house type
    """

    def __init__(self,
                 path="../data/valeursfoncieres-2021.txt",
                 sep="|",
                 chunksize=100000):
        self.path = path
        self.sep = sep
        self.chunksize = chunksize

    def read_csv(self, filtering_column='Code type local', filter=1):
        """ pass option on which column to filter and filter value
        if several filter value, pass the as a list"""
        iter_csv = pd.read_csv(self.path,
                               sep=self.sep,
                               iterator=True,
                               chunksize=self.chunksize,
                               low_memory=False)
        self.df = pd.concat(
            [chunk[chunk[filtering_column].isin(filter)] for chunk in iter_csv])
        return self.df

    def enrichissement_coordinates(self, df):
        pass


In [ ]:
df_tot[df_tot.index.isin([71,72,73,74])]

In [2]:
df2= get_data().read_csv(filter=[1,3])

In [ ]:
# code pour voir comment est rempli pythonpath
#import sys
#print(sys.path)

In [3]:
#from house_prediction_package.data import get_data
import pandas as pd
import numpy as np
from datetime import datetime
from more_itertools import chunked

class preprocessing :

    def __init__(self,df) :
        # self.df = get_data().read_csv()
        self.df = df

    def clean_columns(self,
                      columns=[
                          'Code service CH', 'Reference document',
                          '1 Articles CGI', '2 Articles CGI', '3 Articles CGI',
                          '4 Articles CGI', '5 Articles CGI', 'No Volume',
                          'Identifiant local'
                      ]):
        """ drop useless columns
        Customisation of columns to drop must be entered as a list
        """
        # suppression of 100% empty columns - these columns are officially not completed in this db
        self.df = self.df.drop(columns,axis=1)
        # suppression of columns poorly completed
        columns_to_drop = [column for column in self.df.columns if ((self.df[column].isnull().value_counts().sort_index()[0]/self.df.shape[0])*100) < 2 ]
        self.df= self.df.drop(columns_to_drop,axis=1)
        # suppression of nan value on target variable
        self.df= self.df.dropna(subset='Valeur fonciere')
        # pre processing avant groupby mais attention sortir valeures foncieres avant de mettre en POO
        ob_columns= self.df.dtypes[self.df.dtypes == 'O'].index
        #num_columns= self.df.dtypes[self.df.dtypes == ''].index
        for column in ob_columns :
            self.df[column]=self.df[column].replace(np.nan,'',regex=True)
        #à adapter in v2
        self.df[[
                'Surface terrain', 'Surface reelle bati',
                'Nombre pieces principales', 'Surface Carrez du 1er lot'
        ]] = self.df[[
                'Surface terrain', 'Surface reelle bati',
                'Nombre pieces principales', 'Surface Carrez du 1er lot'
            ]].apply(pd.to_numeric, errors='coerce')
        #drop duplicates
        #self.df = self.df.drop_duplicates().reset_index(drop= True)
        # by returning self, we can do method chaining like preprocessing(df).clean_columns().create_identifier()
        return self

    def create_identifier(self) :
        """ Create a 'unique' identifier allowing us to group several lines corresponding to a unique transaction
        """
        variables_to_clean = [
            "Code departement", "Code commune", "Prefixe de section",
            "Section", "No plan"
            ]
        size_variables= [2,3,3,2,4]
        for i,j in zip(variables_to_clean,size_variables):
            chunked_data = chunked(self.df[i], 10000, strict=False)
            values = {"Prefixe de section": '000'}
            self.df= self.df.fillna(value=values)
            if i == "Prefixe de section" :
                self.df[i] = self.df[i].apply(str).apply(lambda x: x[:3])
            new_variable = [
                str(value).zfill(j) for sublist in list(chunked_data)
                for value in sublist
            ]
            self.df[f"clean_{i.replace(' ','_').lower()}"] = new_variable
            self.df= self.df.drop([i],axis=1)
        self.df["parcelle_cadastrale"] = self.df[[
            "clean_code_departement", "clean_code_commune", "clean_prefixe_de_section",
            "clean_section", "clean_no_plan"]].apply(lambda x: "".join(x), axis=1)
        self.df["parcelle_cad_section"]=self.df["parcelle_cadastrale"].str[:10]
        self.df = self.df.drop([
            "clean_prefixe_de_section", "clean_section", "clean_no_plan"
        ], axis = 1)
        return self

    def aggregate_transactions(self):
        self.df = self.df.groupby(["parcelle_cad_section","Date mutation","Valeur fonciere"], as_index= False).apply(lambda x : pd.Series({
            'B_T_Q' : x['B/T/Q'].max()
            ,'type_de_voie': x['Type de voie'].max()
            ,'voie': x['Voie'].max()
            ,'code_postal': x['Code postal'].max()
            ,'commune': max(x['Commune'])
            ,'clean_code_departement': x['clean_code_departement'].max()
            ,'clean_code_commune': max(x['clean_code_commune'])
            ,'surface_carrez_lot_1' :  x['Surface Carrez du 1er lot'].sum()/(x['Surface reelle bati'].count()/(x['Surface reelle bati'].count()/x['Nature culture'].nunique()))
            ,'Nb_lots': x[('Nombre de lots')].max()
            ,'surface_terrain' : x['Surface terrain'].sum()/(x['Surface terrain'].count()/x['Surface terrain'].nunique()) if int(x['Surface terrain'].nunique()) > 1 and int(x['Nature culture'].nunique()) >1 else x['Surface terrain'].max()
            ,'surface_reelle_bati' : x['Surface reelle bati'].sum()/(x['Surface reelle bati'].count()/(x['Surface reelle bati'].count()/x['Nature culture'].nunique()))
            ,'nb_pieces_principales' : x['Nombre pieces principales'].sum()/(x['Nombre pieces principales'].count()/(x['Surface reelle bati'].count()/x['Nature culture'].nunique()))
            ,'dependance' : x['Type local'].unique()
            ,'main_type_terrain' : x['Nature culture'].max()
            ,'parcelle_cadastrale': x['parcelle_cadastrale'].max()}))
        #drop rows with only dependances transactions as we focus on houses
        self.df = self.df[self.df.dependance.apply(lambda x: x.all() != 'Dépendance')].reset_index(drop=True)
        self.df[['Maison',
                 'Dependance']] = pd.DataFrame(self.dependance.tolist(), index = self.df.index)
        return self.df

    # to do : function calling enrichissement from data

    def feature_generation (self):
        # convert the 'Date' column to datetime format
        self.df['Date_YYYY-MM'] = pd.to_datetime(
            self.df['Date mutation']).dt.to_period('M')
        self.df= self.df.drop(['Date mutation'], axis = 1)
        ## attention à ne faire qu'après avoir enrichi avec variables insee
        dict_type_voie = dict()
        for value in self.df['Type de voie'].value_counts()[self.df['Type de voie'].value_counts()<300 ].index.values :
            dict_type_voie[value] = 'Autres'
        self.df=self.df.replace({'type_voie' : dict_type_voie}).rename(columns = {'Type de voie':'type_voie'})
        return self.df


In [ ]:
df2 = preprocessing(df2).clean_columns().create_identifier().aggregate_transactions()

In [ ]:
df2.to_csv("aggregatedfile.csv", sep='|', encoding="utf-8") 

In [ ]:
df.columns

In [ ]:
df_filtered = df[(df['parcelle_cadastrale']== '01289000AC0176') | (df['parcelle_cadastrale']== '013500000C1248')| (df['parcelle_cadastrale']== '01195000AD0050')|(df['parcelle_cadastrale'] == '013500000C1248')| (df['parcelle_cadastrale'] == '01344000AC0181') |(df.index.isin([71,72,73,74]))]

In [ ]:
df_filtered

# feature importance

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
# si group by parcelle_cad_section et date et valeur fonciere alors même transaction 
#ensuite agrégats différent en fonction 
# si superficie surface reelle bati diff alors plusieurs maisons 
si value_count type local superieur à 1 alors différents locaux 
    agregats suivants : 
        somme surface reelle bati 
        somme nb pieces principales 
        si meme parcelle cadastrale alors max surface terain
        sinon somme surface terrain 
si value_count type local == 1 et que value_count nature culture superieur à 1 alors 
    agregats suviants 
        max surface reelle bati
        max nb pieces principales 
        somme surface terrain 
si value_count type local == 1 et que value_count surface reelle bati superieur à 1 alors :
    agregats suivants 
        somme surface reelle bati
        somme pieces principales 
        max surface terrain 
        
créer une colonne par type local 
     y positionner les value counts de type local 

In [ ]:
#utilisation de dask df pr optimiser le process car on va ajouter les dépendancess 
#find relative frequency of each team name in DataFrame
df.groupby('team').apply(lambda x: x['team'].count() / df.shape[0])

Pour répondre à la problématique d'affichage suivante dansle fichier dvf : 
"Quand une disposition comporte plusieurs locaux ou plusieurs natures de culture, le fichier de restitution comporte autant de lignes qu’il y a de locaux ou de nature de culture concernés par la mutation.

Ainsi, pour une même publication, il peut y avoir 1 à n ligne(s) de restitution. Les données génériques (ainsi que le prix) sont alors répétées sur chaque ligne. 

Cas particulier : dans le cas où n locaux sont construits sur un terrain comportant p natures de cultures différentes, le fichier de restitution comportera (n x p) lignes. En effet, dans ce cas, il n’est pas possible de ventiler automatiquement les locaux par nature de culture, alors le fichier présente toutes les combinaisons possibles.

Par exemple, si 2 locaux L1 et L2 sont construits sur une parcelle comportant 2 natures de culture N1 et N2 (sol et jardin), alors il y aura 4 lignes :
* L1 – N1
* L2 – N2
* L1 – N2
* L2 – N1"

Nous avons opéré en plusieurs étapes :
1. la premiere recréer les pacerelles cadastrales
 
2. la seconde créer des agrégats conditionnés avec des fonctions lambdas. la complexité de l'agrégation réside dans le fait que les aggrégations différent par variable et la typologie d'aggrégation de certaines variables dépendra des données présentes dans d'autres colonnes. si 2 lignes avec une maison mais sur deux terrains, on doit prendre la valeur de superficie de la maison sans la sommer et additionner les superficies terrains. toutefois certaines transactions auront plusieurs maisons et dans ce cas, il faut additionner les superficies maisons.

Les possibilités techniques qui s'offrent à nous sont : 
  *  apply -- Apply function func group-wise and combine the results together.
  *  aggregate -- Apply aggregate function to the GroupBy object.
  *  pipe --    Apply function to the full GroupBy object instead of to each group.
  *  transform -- Apply function column-by-column to the GroupBy object.
  
  Ensuite on drop les dépendances vendues seules (sans maison) 



In [ ]:
#preprocessing droping rows with non numeric values in 
new_df['Type local'] = new_df['Type local'].replace(np.nan, 'Non_spécifié')
new_df= new_df.drop_duplicates()

In [ ]:
# pre processing avant groupby mais attention sortir valeures foncieres avant de mettre en POO
ob_columns= df.dtypes[df.dtypes == 'O'].index
for column in ob_columns : 
    df[column]=df[column].replace(np.nan,'',regex=True)

In [ ]:
df

In [ ]:
df[['Surface terrain','Surface reelle bati','Nombre pieces principales','Surface Carrez du 1er lot']]=df[['Surface terrain','Surface reelle bati','Nombre pieces principales','Surface Carrez du 1er lot']].apply(pd.to_numeric,errors='coerce')

In [ ]:
test = df.groupby(["parcelle_cad_section","Date mutation","Valeur fonciere"], as_index= False).apply(lambda x : pd.Series({
    'B_T_Q' : x['B/T/Q'].max()
    ,'type_de_voie': x['Type de voie'].max()
    ,'voie': x['Voie'].max()
    ,'code_postal': x['Code postal'].max()
    ,'commune': max(x['Commune'])
    ,'clean_code_departement': x['clean_code_departement'].max()
    ,'clean_code_commune': max(x['clean_code_commune'])
    ,'surface_carrez_lot_1' :  x['Surface Carrez du 1er lot'].sum()/(x['Surface reelle bati'].count()/(x['Surface reelle bati'].count()/x['Nature culture'].nunique()))
    ,'Nb_lots': x[('Nombre de lots')].max()
   # ,'v1Surface reelle bati' : x['Surface reelle bati'].max() if x['Type local'].nunique() == 1 and x['Nature culture'].nunique() >1 else x['Surface reelle bati'].sum()
   # ,'v1Nombre pieces principales' : x['Nombre pieces principales'].max() if x['Type local'].nunique() == 1 and x['Nature culture'].nunique() >1 else x['Nombre pieces principales'].sum()
   # ,'v1Surface terrain': x['Surface terrain'].max() if x['Nature culture'].nunique() == 1 or x['parcelle_cadastrale'].nunique() >1 else x['Surface terrain'].sum()
    ,'surface_terrain' : x['Surface terrain'].sum()/(x['Surface terrain'].count()/x['Surface terrain'].nunique()) if int(x['Surface terrain'].nunique()) > 1 and int(x['Nature culture'].nunique()) >1 else x['Surface terrain'].max()
   # ,'v2nb_pieces':  x['Nombre pieces principales'].sum()/(x['Nombre pieces principales'].count()/x['Surface reelle bati'].nunique()) if x['Surface reelle bati'].nunique() > 1 else x['Nombre pieces principales'].max()
   # ,'v2_surface bati' :x['Surface reelle bati'].sum()/(x['Surface reelle bati'].count()/x['Surface reelle bati'].nunique()) if x['Surface reelle bati'].nunique() > 1 else x['Surface reelle bati'].max()
    ,'surface_reelle_bati' : x['Surface reelle bati'].sum()/(x['Surface reelle bati'].count()/(x['Surface reelle bati'].count()/x['Nature culture'].nunique())) 
    ,'nb_pieces_principales' : x['Nombre pieces principales'].sum()/(x['Nombre pieces principales'].count()/(x['Surface reelle bati'].count()/x['Nature culture'].nunique())) 
    ,'dependance' : x['Type local'].unique()
    ,'main_type_terrain' : x['Nature culture'].max()
}))

In [ ]:
df[df['parcelle_cad_section']=='01001000ZH']

In [ ]:
test

In [ ]:
test.dependance.unique()

In [9]:
df=df[df.dependance.apply(lambda x: x != "['Dépendance']")]

In [2]:
df= pd.read_csv("aggregatedfile.csv")

/tmp/ipykernel_4405/3888017884.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv("aggregatedfile.csv")


In [11]:
df

,Unnamed: 0,parcelle_cad_section,Date mutation,Valeur fonciere,B_T_Q,type_de_voie,voie,code_postal,commune,clean_code_departement,clean_code_commune,surface_carrez_lot_1,Nb_lots,surface_terrain,surface_reelle_bati,nb_pieces_principales,dependance,main_type_terrain
0,0,01001000ZE,15/07/2021,"127000,00",NaN,RTE,DU MONT,1400.0,L'ABERGEMENT-CLEMENCIAT,1,1,0.0,0,130.0,42.0,2.0,['Dépendance' 'Maison'],S
1,1,01001000ZH,02/02/2021,"465225,00",NaN,RTE,DE LA FONTAINE,1400.0,L'ABERGEMENT-CLEMENCIAT,1,1,0.0,0,3000.0,188.0,8.0,['Maison'],S
2,2,01001000ZH,05/07/2021,"120000,00",NaN,RUE,DE MUNETVILLE,1400.0,L'ABERGEMENT-CLEMENCIAT,1,1,0.0,0,985.0,75.0,4.0,['Maison'],S
3,3,01001000ZH,19/04/2021,"329000,00",NaN,RUE,DU STADE,1400.0,L'ABERGEMENT-CLEMENCIAT,1,1,0.0,0,1078.0,118.0,5.0,['Dépendance' 'Maison'],S
4,4,01001000ZH,23/06/2021,"198000,00",NaN,RUE,DES MURIERS,1400.0,L'ABERGEMENT-CLEMENCIAT,1,1,0.0,0,755.0,76.0,4.0,['Maison'],S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
808228,808228,974024000A,18/10/2021,"180000,00",NaN,IMP,ETHEVE,97413.0,CILAOS,974,24,0.0,0,936.0,78.0,4.0,['Maison'],S
808229,808229,974024000A,27/04/2021,"220000,00",NaN,CHE,TERRE BLANCHE,97413.0,CILAOS,974,24,0.0,0,10500.0,90.0,4.0,['Maison'],T
808230,808230,974024000A,27/05/2021,"61000,00",NaN,PAS,DES MARCHES,97413.0,CILAOS,974,24,0.0,0,375.0,30.0,4.0,['Dépendance' 'Maison'],S
808231,808231,974024000A,28/05/2021,"148850,00",NaN,RUE,DES GLYCINES,97413.0,CILAOS,974,24,0.0,0,177.0,60.0,6.0,['Maison'],S


In [ ]:
test = df_filtered.groupby(["parcelle_cad_section","Date mutation","Valeur fonciere"], as_index= False).apply(lambda x : pd.Series({
    'Code postal': x['Code postal'].max()
    ,'Commune': max(x['Commune'])
    ,'Surface Carrez du 1er lot' : max(x['Surface Carrez du 1er lot']) if x['Type local'].nunique() == 1 and x['Nature culture'].nunique() >1 else x['Surface Carrez du 1er lot'].sum()
    ,'Nombre de lots': x[('Nombre de lots')].max()
    ,'Surface reelle bati' : x['Surface reelle bati'].max() if x['Type local'].nunique() == 1 and x['Nature culture'].nunique() >1 else x['Surface reelle bati'].sum()
    ,'Nombre pieces principales' : x['Nombre pieces principales'].max() if x['Type local'].nunique() == 1 and x['Nature culture'].nunique() >1 else x['Nombre pieces principales'].sum()
    ,'Surface terrain': x['Surface terrain'].max() if x['Nature culture'].nunique() == 1 or x['parcelle_cadastrale'].nunique() >1 else x['Surface terrain'].sum()
    ,'v2' : x['Surface terrain'].sum()/(x['Surface terrain'].count()/x['Surface terrain'].nunique()) if x['Surface terrain'].nunique() > 1 and x['Nature culture'].nunique() >1 else x['Surface terrain'].max()
    ,'v2nb_pieces':  x['Nombre pieces principales'].sum()/(x['Nombre pieces principales'].count()/x['Surface reelle bati'].nunique()) if x['Surface reelle bati'].nunique() > 1 else x['Nombre pieces principales'].max()
    ,'v2_surface bati' :x['Surface reelle bati'].sum()/(x['Surface reelle bati'].count()/x['Surface reelle bati'].nunique()) if x['Surface reelle bati'].nunique() > 1 else x['Surface reelle bati'].max()
    ,'v3_surface_bati' : x['Surface reelle bati'].sum()/(x['Surface reelle bati'].count()/(x['Surface reelle bati'].count()/x['Nature culture'].nunique())) 
    ,'v3_nb_pieces' : x['Nombre pieces principales'].sum()/(x['Nombre pieces principales'].count()/(x['Surface reelle bati'].count()/x['Nature culture'].nunique())) 
    ,'Type local': x['Type local'].nunique()
    ,'Nature culture' : x['Nature culture'].nunique()
    ,'test' : x['Nature culture'].count()
}))

In [ ]:
# même maison surface reelle bati identique 
df[(df['parcelle_cadastrale'] == '013500000C1248')]
#actions possibles : 
# meme valeur fonciere 
# meêm type local 
# même surface relle bati
 #pas d'info sur 1er lot
# pas d info Nombre de lots 
# même nombre de pieces principales

# différence nature culture (variable texte )
# surface terrain différente en fonction de la parcelle cadastrale 


In [ ]:
# cas ou type local identique mais nature culture différente: 

df[(df['parcelle_cadastrale']== '01289000AC0176') | (df['parcelle_cadastrale']== '013500000C1248')| (df['parcelle_cadastrale']== '01195000AD0050')]
#actions possibles : 
# meme valeur fonciere 
# meme de surface reelle bati 
#pas d'info sur 1er lot
# pas d info Nombre de lots 
#meme nombre pieces principales 
# différente nature culture (variable texte )
# différente surface terrain 


In [ ]:
# cas ou 2 maisons , une dépendance et un terrain :
# repérable par section et date commune 


df[df.index.isin([71,72,73,74])]
#actions possibles : 
# meme valeur fonciere 
# code type lcoal différent pr dépendance absent pour terrain
# différence de surface reelle bati  (0 dépendance et nan pour terrain)
#pas d'info sur 1er lot
# pas d info Nombre de lots 
# différence sur nombre de pieces principales

# différence nature culture (variable texte )
# surface terrain différente en fonction de la parcelle cadastrale 


In [ ]:
test=df_filtered.groupby(["parcelle_cad_section","Date mutation","Valeur fonciere"], as_index= False).agg({
    'Code postal': 'max'
    ,'Commune': 'max'
    ,'Surface Carrez du 1er lot' : lambda x : x.max() if df_filtered['Type local'].nunique() == 1 and df_filtered['Nature culture'].nunique() >1 else x.sum()
    ,'Nombre de lots': 'max'
    ,'Surface reelle bati' : lambda x : x.max() if ['Type local'].nunique() == 1 and ['Nature culture'].nunique() >1 else x.sum()
    ,'Nombre pieces principales' : lambda x : x.max() if df_filtered['Type local'].nunique() == 1 and df_filtered['Nature culture'].nunique() >1 else x.sum()
    ,'Surface terrain' : lambda x : x.max() if df_filtered['Nature culture'].nunique() == 1 or df_filtered['parcelle_cadastrale'].nunique() >1 else x.sum()
    ,'Type local': lambda x: x.value_counts().index[0]
    ,'clean_code_departement' : lambda x: x.value_counts().index[0]
    ,'clean_code_commune' : lambda x: x.value_counts().index[0]
    ,'Nature culture' : 'nunique'
    ,'Nature mutation' : 'count'})


In [ ]:
 
df_filtered[(df_filtered['parcelle_cadastrale'] == '01289000AC0176')].groupby(["parcelle_cad_section","Date mutation","Valeur fonciere"], as_index= False).agg({
    'Type local':lambda x : x.nunique()
    ,'Nombre pieces principales' : lambda x : x.max() if prout['Type local'].nunique() == 1 and prout['Nature culture'].nunique() >1 else x.sum()
    ,'Nature culture':lambda x : x.nunique()})
    


In [ ]:
# convert the 'Date' column to datetime format \n
df_maison['Date mutation']= pd.to_datetime(df_maison['Date mutation'],format= "%d/%m/%Y")
#df_maison= df_maison.drop(['Date mutation'], axis = 1)

In [ ]:
df_maison.columns

In [ ]:
# supression des colonnes innutiles avant chargement dans db 
df_maison=df_maison.drop(['Code service CH', 'Reference document', '1 Articles CGI',
'2 Articles CGI', '3 Articles CGI', '4 Articles CGI', '5 Articles CGI','No Volume','Identifiant local'], axis =1 )

## Performances

On a des problemes de performances 1h pr ingérer 1/25 des données brutes. 2 MIllions de lignes et plus de 30 colonnes
avant d'envoyer en base, on va essayer de nettoyer le df (suppression des colonnes inutiles, des doublons, des lignes vides)
regroupement des lignes correspondant à 1 seule transaction- création d'un id et group by 
recentrer le projet sur les mventes de maisons

In [ ]:
## à vérifier si cela fonctionne
engine = create_engine('sqlite:///../data/transactions.sqlite', echo = True)
Session = sessionmaker(bind=engine)
Base = declarative_base()

In [ ]:
# à corriger en reprenant typo de l'exercice foot
class Transactions_maisons(Base):
        __tablename__ = 'dvf_maisons'

        id = Column(Integer, primary_key=True)
        no_disposition = Column(Integer)
        date_mutation = Column(Date)
        nature_mutation = Column(String)
        valeur_fonciere = Column(Integer)
        no_voie = Column(Integer)
        b_t_q = Column(String)
        type_voie = Column(String)
        code_voie = Column(String)
        voie = Column(String)
        code_postal = Column(Integer)
        commune = Column(String)
        code_departement = Column(String)
        code_commune = Column(String)
        prefixe_section = Column(String)
        section = Column(String)
        no_plan = Column(Integer)
        lot_1 = Column(String)
        surface_carrez_1er_lot = Column(Integer)
        lot_2 = Column(String)
        surface_carrez_2eme_lot = Column(Integer)
        lot_3 = Column(String)
        surface_carrez_3eme_lot = Column(Integer)
        lot_4 = Column(String)
        surface_carrez_4eme_lot = Column(Integer)
        lot_5 = Column(String)
        surface_carrez_5eme_lot = Column(Integer)
        nombre_lots = Column(Integer)
        code_type_local = Column(Float)
        type_local = Column(String)
        surface_reelle_bati = Column(Integer)
        nombre_pieces_principales = Column(Integer)
        nature_culture = Column(String)
        nature_culture_speciale = Column(String)
        surface_terrain = Column(Integer)
        
        def __init__(self, no_disposition, date_mutation, nature_mutation, valeur_fonciere, no_voie,
                     b_t_q, type_voie, code_voie, voie, code_postal, commune, code_departement, 
                     code_commune, prefixe_section, section, no_plan, lot_1, surface_carrez_1er_lot, 
                     surface_carrez_2eme_lot, surface_carrez_3eme_lot, surface_carrez_4eme_lot, 
                     surface_carrez_5eme_lot, lot_2, lot_3, lot_4, lot_5, nombre_lots, code_type_local, 
                     type_local, surface_reelle_bati, nombre_pieces_principales, nature_culture, nature_culture_speciale,
                     surface_terrain ):
            self.no_disposition = no_disposition 
            self.date_mutation = date_mutation 
            self.nature_mutation = nature_mutation
            self.valeur_fonciere = valeur_fonciere 
            self.no_voie = no_voie
            self.b_t_q = b_t_q 
            self.type_voie = type_voie 
            self.code_voie = code_voie 
            self.voie = voie 
            self.code_postal = code_postal
            self.commune = commune
            self.code_departement = code_departement
            self.code_commune = code_commune
            self.prefixe_section = prefixe_section
            self.section = section
            self.no_plan = no_plan
            self.lot_1 = lot_1
            self.surface_carrez_1er_lot = surface_carrez_1er_lot
            self.lot_2 = lot_2
            self.surface_carrez_2eme_lot = surface_carrez_2eme_lot
            self.lot_3 = lot_3
            self.surface_carrez_3eme_lot = surface_carrez_3eme_lot
            self.lot_4 = lot_4
            self.surface_carrez_4eme_lot = surface_carrez_4eme_lot
            self.lot_5 = lot_5
            self.surface_carrez_5eme_lot = surface_carrez_5eme_lot
            self.nombre_lots = nombre_lots
            self.code_type_local = code_type_local
            self.type_local = type_local
            self.surface_reelle_bati = surface_reelle_bati
            self.nombre_pieces_principales = nombre_pieces_principales
            self.nature_culture = nature_culture
            self.nature_culture_speciale = nature_culture_speciale
            self.surface_terrain =surface_terrain 

In [ ]:
# delete table
#Transactions_maisons.__table__.drop(engine)"

In [ ]:
# 2 - generate database schema
Base.metadata.create_all(engine)
# 3 - create a new session
session = Session()

In [ ]:
n = 2000  #chunk row size
list_df = [df_maison[i:i+n] for i in range(0,df_maison.shape[0],n)]
# reassemblage by pd.concat possible mais on s'en fiche car on va fonctionner sur des'petits df' 
#pour enrichissement puis insertion en bdd

In [ ]:
test = 'ok'
start_time = datetime.now()
for j in range(len(list_df)):
    if test == 'ok':
        for i,transaction in list_df[j].iterrows():
            session= Session()
            new_transaction = Transactions_maisons(transaction.values[0],transaction.values[1],transaction.values[2],transaction.values[3],transaction.values[4],
                                                      transaction.values[5],transaction.values[6],transaction.values[7],transaction.values[8],transaction.values[9],
                                                      transaction.values[10],transaction.values[11],transaction.values[12],transaction.values[13],transaction.values[14],
                                                      transaction.values[15],transaction.values[16],transaction.values[17],transaction.values[18],transaction.values[19],
                                                      transaction.values[20],transaction.values[21],transaction.values[22],transaction.values[23],transaction.values[24],
                                                      transaction.values[25],transaction.values[26],transaction.values[27],transaction.values[28],transaction.values[29],
                                                      transaction.values[30],transaction.values[31],transaction.values[32],transaction.values[33])
            session.add(new_transaction)
            session.commit()
            session.close()
    else  :
        break
    test= input(f"iteration {j}, pour passer à l'itération {j+1} taper ok  : ")
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

In [ ]:
#integré à la boucle 
#session.commit()
session.close()

In [ ]:
session = Session()

In [ ]:
rows = session.query(Transactions_maisons).count()
rows